# LangChain Expression Language (LCEL)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables (expects OPENAI_API_KEY to be set in your environment or .env)
_ = load_dotenv(find_dotenv())
assert os.environ.get("OPENAI_API_KEY"), "OPENAI_API_KEY is not set"


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


## Simple Chain

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(model="gpt-5-mini", temperature=0)
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

'What do you call a bear with no teeth? A gummy bear!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


In [7]:
vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    collection_name="lcel_demo",
)
retriever = vectorstore.as_retriever()


In [8]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [9]:
retriever.invoke("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain_core.runnables import RunnableMap


In [12]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [13]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [14]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [15]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI tool calling

In [16]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool

class WeatherSearchInput(BaseModel):
    airport_code: str = Field(..., description="The airport code to get the weather for")

@tool(args_schema=WeatherSearchInput)
def weather_search(airport_code: str) -> str:
    """Search for weather given an airport code."""
    # Dummy implementation for teaching/demo purposes
    return f"The weather at {airport_code.upper()} is sunny."

class SportsSearchInput(BaseModel):
    team_name: str = Field(..., description="The sports team to search for")

@tool(args_schema=SportsSearchInput)
def sports_search(team_name: str) -> str:
    """Search for news of recent sport events."""
    # Dummy implementation for teaching/demo purposes
    return f"Latest result: {team_name} won their last game."


In [17]:
prompt = ChatPromptTemplate.from_messages([("human", "{input}")])

model = ChatOpenAI(model="gpt-5-mini", temperature=0).bind_tools([weather_search, sports_search])
runnable = prompt | model


In [18]:
runnable.invoke({"input": "What is the weather at SFO? Use the weather_search tool."})

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 182, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsccexifcrIbCTRimPLOWDTgwKIPY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7168-3095-7d10-9c8c-b8e39b9a01af-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'SFO'}, 'id': 'call_oFfm0c4sz2VC3nQ6qpNxKvH8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 25, 'total_tokens': 207, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
runnable.invoke({"input": "How did the Patriots do in their most recent game? Use the sports_search tool."})

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 185, 'total_tokens': 276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csccg4hjDnzHOdwJJIoGENuGh4iXd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7168-363a-7ee2-bf99-8da8c9c4449d-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': 'New England Patriots'}, 'id': 'call_r01Cw1eU4ZTyPHnV34gNxlzP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 91, 'total_tokens': 276, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

## Fallbacks and JSON reliability patterns

This section shows three ways to make “LLM → JSON” workflows reliable:

1) **Fallbacks**: if parsing fails, retry with a stricter chain.
2) **Structured outputs (schema-guided)**: guide the model to output JSON matching a schema (no fallback needed).
3) **JSON repair**: insert a repair stage between `StrOutputParser()` and `json.loads`.

All examples below are **self-contained**: they define any variables they need inside this section,
so they don’t depend on cells later in the notebook.


In [20]:
import json
from typing import List

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda

# Use the notebook's `model` if it already exists, otherwise create a local one.
try:
    fallback_model = model  # noqa: F821
except NameError:
    fallback_model = ChatOpenAI(model="gpt-5-mini", temperature=0)


### A) Fallbacks (a real recovery example)

**Goal:** parse JSON reliably even when the model sometimes includes extra text.

- **Primary chain** is *designed to fail* JSON parsing by asking for an explanation **before** the JSON.
  That makes the output invalid JSON for `json.loads`.
- **Fallback chain** repeats the task with a stricter instruction: **return ONLY JSON**.
- `with_fallbacks([fallback_chain])` runs the fallback automatically when the primary chain throws an exception.


In [21]:
# Primary prompt intentionally produces non-JSON text + JSON => json.loads will fail.
bad_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", """Write 3 short poems about bears.
First, explain briefly what you will do.
Then provide a JSON object with key: poems (array of 3 strings).""")
])

primary_chain = bad_prompt | fallback_model | StrOutputParser() | json.loads

# Fallback prompt forces ONLY JSON.
good_prompt = ChatPromptTemplate.from_messages([
    ("system", "Return ONLY valid JSON. No extra text. No markdown fences."),
    ("user", """Write 3 short poems about bears.
Return a JSON object with key:
- poems: array of 3 strings""")
])

fallback_chain = good_prompt | fallback_model | StrOutputParser() | json.loads

final_chain = primary_chain.with_fallbacks([fallback_chain])

result_fallback = final_chain.invoke({})
result_fallback


{'poems': ['Paws press on soft moss\nAmber breath fogs the cold air\nRiver remembers',
  'Cub tumbles in sun\nDandelion crowns, sticky paws\nNap in a honeyed dream',
  'Claw marks on the cliff\nYears counted in salmon bones\nNight sky tucks him home']}

Optional: show the exception that triggers the fallback (for learning).

In [22]:
try:
    primary_chain.invoke({})
except Exception:
    import traceback
    traceback.print_exc()


Traceback (most recent call last):
  File "/var/folders/l9/3b84hrtx38sd0981n54901g80000gn/T/ipykernel_1808/1405841695.py", line 2, in <module>
    primary_chain.invoke({})
    ~~~~~~~~~~~~~~~~~~~~^^^^
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 3151, in invoke
    input_ = context.run(step.invoke, input_, config)
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 4880, in invoke
    return self._call_with_config(
           ~~~~~~~~~~~~~~~~~~~~~~^
        self._invoke,
        ^^^^^^^^^^^^^
    ...<2 lines>...
        **kwargs,
        ^^^^^^^^^
    )
    ^
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 2058, in _call_with_config
    context.run(
    ~~~~~~~~~~~^
        call_func_with_variable_args,  # type: ignore[arg-type]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### B) Without fallbacks: structured outputs (schema-guided)

**How it works**
- Define the output shape using a **Pydantic v2** model.
- `JsonOutputParser` generates *format instructions* that you inject into the prompt.
- The model is guided to output JSON matching the schema.
- The parser validates and returns structured data.

This typically avoids JSON parsing failures because the model is strongly constrained to produce valid JSON.


In [23]:
class Poems(BaseModel):
    poems: List[str] = Field(..., description="Three short poems about bears")

structured_parser = JsonOutputParser(pydantic_object=Poems)

structured_prompt = ChatPromptTemplate.from_messages([
    ("system", "Follow the format instructions exactly."),
    ("user", """Write 3 short poems about bears.

{format_instructions}
""")
]).partial(format_instructions=structured_parser.get_format_instructions())

structured_chain = structured_prompt | fallback_model | structured_parser

result_structured = structured_chain.invoke({})
result_structured


{'poems': ['Forest hush, brown giant\npaws press the earth, slow and sure\nhoney-sweet dawn.',
  'Winter curled, white breath\nsleep thick as the fallen snow\ndreams of rivers.',
  "Cub tumbles in sun\nmother's shadow watches close\nlaughter in the grass."]}

### C) JSON repair step between `StrOutputParser()` and `json.loads`

**How it works**
- Keep a generation chain that might emit messy text.
- Attempt `json.loads(text)`.
- If it fails, call a *repair chain* that rewrites the text into **valid JSON only**.
- Parse the repaired JSON.

This adds robustness when you can’t fully control the upstream prompt.
Tradeoff: it may require a second model call when repair is needed.


In [24]:
repair_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a JSON repair assistant. Return ONLY valid JSON. No extra text. No markdown fences."),
    ("user", """Fix the following text into valid JSON that matches this schema:

Schema:
- poems: array of 3 strings

Text to repair:
{text}
""")
])

repair_chain = repair_prompt | fallback_model | StrOutputParser()

def parse_or_repair(text: str):
    try:
        return json.loads(text)
    except Exception:
        repaired = repair_chain.invoke({"text": text})
        return json.loads(repaired)

json_repair_step = RunnableLambda(parse_or_repair)

# A "bad" generator likely to include extra non-JSON text.
bad_generation = bad_prompt | fallback_model | StrOutputParser()

repaired_chain = bad_generation | json_repair_step

result_repaired = repaired_chain.invoke({})
result_repaired


{'poems': ['Honey-scented dusk\nheavy paws print the soft earth-\nmoon tucks in the den',
  'A cub flops, surprised,\nclumsy joy unspools in green,\nwind teaches it to roar',
  'River-silvered eyes,\nan old bear remembers seasons—\nstones keep his stories']}

### Combine with `with_fallbacks` and invoke

If `primary_chain` errors, LangChain automatically runs `fallback_chain` with the same input.


In [25]:
final_chain = primary_chain.with_fallbacks([fallback_chain])

result = final_chain.invoke({})
result


{'poems': ['Mountain shadow moves\nHeavy breath in berry dusk\nWinter hush listens',
  'She pads through thawing streams,\nBroad shoulders brushing river light.\nA thumb of honey on her tongue,\nMemory of sleep in the soil.',
  'Cub rolls like a barrel\nChasing leaves and summer sun;\nMother laughs in the pines.']}

### Optional: show the error that triggers the fallback

This cell shows the exception thrown by the primary chain.


In [26]:
try:
    primary_chain.invoke({})
except Exception:
    import traceback
    traceback.print_exc()


Traceback (most recent call last):
  File "/var/folders/l9/3b84hrtx38sd0981n54901g80000gn/T/ipykernel_1808/1405841695.py", line 2, in <module>
    primary_chain.invoke({})
    ~~~~~~~~~~~~~~~~~~~~^^^^
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 3151, in invoke
    input_ = context.run(step.invoke, input_, config)
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 4880, in invoke
    return self._call_with_config(
           ~~~~~~~~~~~~~~~~~~~~~~^
        self._invoke,
        ^^^^^^^^^^^^^
    ...<2 lines>...
        **kwargs,
        ^^^^^^^^^
    )
    ^
  File "/Users/markberman/miniconda3/envs/langchain/lib/python3.13/site-packages/langchain_core/runnables/base.py", line 2058, in _call_with_config
    context.run(
    ~~~~~~~~~~~^
        call_func_with_variable_args,  # type: ignore[arg-type]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Interface

In [27]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(model="gpt-5-mini", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [28]:
chain.invoke({"topic": "bears"})

'What do you call a bear with no teeth? A gummy bear.'

In [29]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['What do you call a bear with no teeth? A gummy bear.',
 "Why are frogs so happy? Because they eat whatever's bugging them."]

In [30]:
for t in chain.stream({"topic": "bears"}):
    print(t)


What
 do
 you
 call
 a
 bear
 with
 no
 teeth
?
 A
 gummy
 bear
.





In [31]:
response = await chain.ainvoke({"topic": "bears"})
response

'What do you call a bear with no teeth? A gummy bear.'